In this kernal you will find detailed EDA,where each features (countinuous numerical variable)has been converted into categroical variable to obtain more sense 
from the data. Changing into categorical has help to obtain more sense and better accuracy from the model. EDA and feature engineering are two  most improtant aspect of machine Learning, if you cannot understand the trend of the data, machine can also not able to find any trend, thus leading to low accuracy of the model.

Please go through the kernal and if you like it add comments and upvote.


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing
import codecs# this is used for file operations 
import random as r
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
%matplotlib inline
from sklearn.metrics import r2_score
from sklearn import metrics

In [ ]:
data = pd.read_csv("/kaggle/input/insurance/insurance.csv")
data.head()

In [ ]:
data.shape

In [ ]:
data.info()


In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
print("Find most important features relative to target")
corr = data.corr()
corr.sort_values(["charges"], ascending = False, inplace = True)
corr
print(corr['charges'])

# Data Visualization

In [ ]:
sns.distplot(data['charges'])

In [ ]:
data['charges'].skew()

The above distribution tells us the charge is not normally distributed, It has high skewness. We will deal with it later. Also most of charges are below 50k

# SEX

In [ ]:
sns.countplot(data['sex'])

Count of male and female is more or less equal

In [ ]:
g=sns.FacetGrid(data,col='sex')
g=g.map(sns.distplot,"charges")


In [ ]:
sns.catplot(y='charges',x='sex',data=data,kind="swarm")

It does not look there is any difference in the charging based on sex of an individual. We will explore further with other variable.
Till now insurance does not depend if you are male or female.

# SMOKER

In [ ]:
sns.catplot(y='charges',x='sex',col='smoker',data=data,kind="swarm")

In [ ]:
sns.catplot(y='charges',x='sex',col='smoker',data=data,kind="box")

In [ ]:
sns.catplot(y='charges',x='sex',col='smoker',data=data,kind="violin")

In [ ]:
data[(data['smoker']=='yes')&(data['sex']=='male')]['charges'].mean()

In [ ]:
data[(data['smoker']=='yes')&(data['sex']=='female')]['charges'].mean()

From the diagram it is clear that smoking has high correlation with charges, if you smoke , then insurance paid will be high irrespective
of gender. Further in the smoking categroy the mean of male smoker is higher than female smoker, need to compare with other variable like
age and bmi to know why are they charged more than female.

# Age

In [ ]:
sns.distplot(data['age'])

In [ ]:
sns.scatterplot(x='age', y='charges',data=data)

In [ ]:
data['age'].describe()

From the data it is visible that the minimum age is 18 and the maximum age is 64. So if we convert this numerical data into categorical
data we can get better insight of the data.

SO lets convert the age into: 18yrs-26 yrs as young , 27yrs-40yrs as adult,41yrs to 55yrs as senior adult and 56yrs to 64yrs as Elder

In [ ]:
data['age_cat']=np.NAN
lst = [data]

for col in lst:
    col.loc[(col['age'] >= 18) & (col['age'] <= 26), 'age_cat'] = 'Young'
    col.loc[(col['age'] > 26) & (col['age'] <= 40), 'age_cat'] = 'Adult'
    col.loc[(col['age'] > 40) & (col['age'] <= 55), 'age_cat'] = 'Senior Adult'
    col.loc[col['age'] > 55, 'age_cat'] = 'Elder'

In [ ]:
data['age_cat'].value_counts()

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='bar')

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='box')

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='violin')

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='swarm')

From the above we can now easily see the corelation(ordinal) between age and charges, as the age increase insurance charge increase. But each
group has some high values need to understand the reason, using other features.

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='box',col='smoker')

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='box',col='smoker',hue='sex')

In [ ]:
sns.catplot(x='age_cat', y='charges',data=data,kind='violin',col='smoker',hue='sex')

In [ ]:
data[(data['age_cat']=='Adult')&(data['smoker']=='yes')&(data['sex']=='male')]['charges'].describe()

In [ ]:
data[(data['age_cat']=='Adult')&(data['smoker']=='yes')&(data['sex']=='female')]['charges'].describe()

So from this we can confirm that non smoker of any age has low insurance charges in comparison to smokers.

1)In non smoker category, Males has slightly less insurance cost in comparison to female.
2)in smoker category, mean cost of male are higher than female.


In [ ]:
data.head()

# BMI

In [ ]:
sns.scatterplot(x='bmi',y='charges',data=data)

In [ ]:
sns.distplot(data['bmi'])

In [ ]:
data['bmi'].describe()

If we see a BMI chart, there is 5 classes, 15 to 18 is underweight , 19 to 24 is healthy ,25 to 29 is overweight ,30 to 39 
is obese and greater than 40 is extremly obese.So we will divide our data into that categories.


In [ ]:
data.loc[(data['bmi']>= 15)&(data['bmi']<19), 'bmi_cat'] = 'underweight'
data.loc[(data['bmi']>= 19)&(data['bmi']<25), 'bmi_cat'] = 'healthy'
data.loc[(data['bmi']>= 25)&(data['bmi']<30), 'bmi_cat'] = 'overweight'
data.loc[(data['bmi']>= 30)&(data['bmi']<40), 'bmi_cat'] = 'obese'
data.loc[(data['bmi']>= 40), 'bmi_cat'] = 'ext_obese'

In [ ]:
data['bmi_cat'].value_counts()

In [ ]:
data['bmi_cat'].unique()

In [ ]:
sns.catplot(x='bmi_cat',y='charges',kind='bar',data=data)

In [ ]:
sns.catplot(x='bmi_cat',y='charges',kind='box',data=data)

In [ ]:
sns.catplot(x='bmi_cat',y='charges',kind='swarm',data=data)

In [ ]:
sns.catplot(x='bmi_cat',y='charges',col='smoker',kind='box',data=data)

In [ ]:
sns.catplot(x='bmi_cat',y='charges',col='age_cat',hue='smoker',kind='box',data=data)

Lets analyzse the above plots. In all age category on thing is common, if you are obese or more  and you smoke the expense will
be very high.
The same is applicabe for overweight and healthy but the ratio of increase is lesser than obese.
Further with increasein age, expense increases as we have seen earlier.

# CHILDREN

In [ ]:
sns.countplot(data['children'])

In [ ]:
sns.stripplot(x="children", y="charges", data=data, size = 5, jitter = True)

In [ ]:
data['children'].value_counts()

In [ ]:
data['children']=data['children'].map({0:'0',1:'1',2:'2',3:'3+',4:'3+',5:'3+'})

In [ ]:
data['children'].value_counts()

In [ ]:
sns.catplot(x='children',y='charges',data=data,kind='bar')

Lets see if people tend to quit smoking or dont smoke if they have children

In [ ]:
sns.countplot(x='children',hue='smoker',data=data)

In [ ]:
data['smoker'].value_counts()

No children present or not people do smoke

In [ ]:
data.head()

Lets see if you have more children then do you get good time to take care of yourselve, eat well, do workout etc..or you dont get time and 
time and get overweight.

In [ ]:
sns.catplot(x='children',y='bmi',data=data,kind='swarm')

In [ ]:
sns.catplot(x='children',y='bmi',hue='smoker',data=data,kind='box')

In [ ]:
sns.catplot(x='children',y='charges',data=data,kind='bar')

It looks like having children dont affect your BMI. However with increase in no of children medical expense increases. but if you see above distribution we can safely say having 0 or 1 child expnse is more or less equal, similarly having 2 or more child expense will be similar. So lets disribute into two part. 'less' if no of children is less than equal to 1 and 'more; if it is greater than equal to 2.

In [ ]:
data['child_cat']=np.NAN
data.loc[(data['children']=='0')|(data['children']=='1'), 'child_cat'] = 'less'
data.loc[(data['children']=='2')|(data['children']=='3+'), 'child_cat'] = 'more'

In [ ]:
data.head()

# REGION

In [ ]:
data['region'].value_counts()

Let see the distribution of smoker based on region

In [ ]:
sns.countplot(x='region',hue='smoker',data=data)

Southeast has the maximum no of smoker followed by northeast , thus the charges will also be high for southeast followed by 
northeast.


In [ ]:
sns.catplot(x='region',y='charges',hue='smoker',data=data,kind='box')

In [ ]:
sns.catplot(x='region',y='charges',hue='smoker',data=data,kind='swarm')

As we thought since southeast had maximum no of smoker followed by northeast, charges are higher for southeast.

In [ ]:
sns.catplot(x='region',y='charges',col='smoker',data=data,kind='bar')

In [ ]:
sns.catplot(x='region',y='charges',col='bmi_cat',hue='smoker',data=data,kind='bar')

In [ ]:
sns.catplot(x='region',y='charges',data=data,kind='bar')

We cannot find any trend in charges based on region, when we are consider all the factors. thus we will one hot code this feature.

In [ ]:
data1=data.copy()

# FEATURE ENGINEERING

In [ ]:
data1.head()

In [ ]:
data1=data1.drop(['age','bmi','children'],axis=1)


In [ ]:
data1['age_cat'].unique()

In [ ]:
data1['bmi_cat'].unique()

In [ ]:
data1.info()

In [ ]:
data1['age_cat']=data1['age_cat'].map({'Young':0, 'Adult':1, 'Senior Adult':2, 'Elder':3})
data1['bmi_cat']=data1['bmi_cat'].map({'underweight':0, 'healthy':1, 'overweight':2, 'obese':3,'ext_obese':4})
data1['child_cat']=data1['child_cat'].map({'less':0, 'more':1})


In [ ]:
#data_hot=data1[['sex','smoker','region','age_cat','bmi_cat','child_cat']]

In [ ]:
#data_hot1 = pd.get_dummies(data_hot)

In [ ]:
#data_final=pd.concat([data_hot1,data1['charges']],axis=1)

In [ ]:
data1.head()

In [ ]:
data1['smoker']=data1['smoker'].map({'no':0, 'yes':1})

In [ ]:
data1.head()

In [ ]:
data_hot = pd.get_dummies(data1['region'])
data_hot1 = pd.get_dummies(data1['sex'])

In [ ]:
data1

In [ ]:
data1=data1.drop(['region','sex'],axis=1)

In [ ]:
data_final=pd.concat([data1,data_hot,data_hot1],axis=1)

In [ ]:
data_final.head()

Now the final part, we will work on charges. Let check the skewness.

# Skewness of dependent variable

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

sns.distplot(data['charges'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(data_final['charges'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('charge distribution')

fig = plt.figure()
res = stats.probplot(data['charges'], plot=plt)
plt.show()

In [ ]:
data_final['charges'] = np.log1p(data_final['charges'] )
y=data_final['charges']

sns.distplot(data_final['charges'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(data_final['charges'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')

fig = plt.figure()
res = stats.probplot(data['charges'], plot=plt)
plt.show()

In [ ]:
data_final.head()

In [ ]:
data_final.shape

In [ ]:
#data_final=data_final.drop(['sex_male','smoker_no','region_southwest','age_cat_Young','bmi_cat_healthy','child_cat_more'],axis=1)

# Modelling

In [ ]:
Data_out=data_final['charges']
input_data=data_final.drop(['charges'],axis=1)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
quad = PolynomialFeatures (degree = 2)
x_quad = quad.fit_transform(input_data)


In [ ]:
R2_Scores = []
models1 = ['Linear Regression' , 'GradientBoosting Regression' ,'DecisionTreeRegressor','SVR','RandomForestRegressor','KNeighbours Regression']

In [ ]:
#data_y = result['Class']
# split the data into test and train by maintaining same distribution of output varaible 'y_true' [stratify=y_true]
X_train, X_test, y_train, y_test = train_test_split(input_data,Data_out,test_size=0.10)
# split the train data into train and cross validation by maintaining same distribution of output varaible 'y_train' [stratify=y_train]
#X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train,test_size=0.10)

In [ ]:
print('Number of data points in train data:', X_train.shape[0])
print('Number of data points in test data:', X_test.shape[0])
#print('Number of data points in cross validation data:', X_cv.shape[0])

In [ ]:
data_final.shape

# Standarization


In [ ]:
# X_train.describe()


y_train= y_train.values.reshape(-1,1)
y_test= y_test.values.reshape(-1,1)
#y_cv= y_cv.values.reshape(-1,1)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)
y_train = sc_X.fit_transform(y_train)
y_test = sc_y.fit_transform(y_test)
#X_cv = sc_X.fit_transform(X_cv)
#y_cv = sc_y.fit_transform(y_cv)

# Gradient Boosting


In [ ]:
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

In [ ]:
clf_pred=clf.predict(X_test)
clf_pred= clf_pred.reshape(-1,1)

In [ ]:
r2=r2_score(y_test, clf_pred)
print('MAE:', metrics.mean_absolute_error(y_test, clf_pred))
print('MSE:', metrics.mean_squared_error(y_test, clf_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, clf_pred)))
print('r2 score:',r2)
R2_Scores.append(r2)

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_test,clf_pred, c= 'brown')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.show()

# DecisionTreeRegressor


In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtreg = DecisionTreeRegressor(random_state = 100)
dtreg.fit(X_train, y_train)

In [ ]:
dtr_pred = dtreg.predict(X_test)
dtr_pred= dtr_pred.reshape(-1,1)

In [ ]:
r2=r2_score(y_test, dtr_pred)
print('MAE:', metrics.mean_absolute_error(y_test, dtr_pred))
print('MSE:', metrics.mean_squared_error(y_test, dtr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, dtr_pred)))
print('r2 score:',r2)

R2_Scores.append(r2)

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_test,dtr_pred,c='green')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.show()

# SVR

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf')
svr.fit(X_train, y_train)

In [ ]:
svr_pred = svr.predict(X_test)
svr_pred= svr_pred.reshape(-1,1)

In [ ]:
r2=r2_score(y_test, svr_pred)
print('MAE:', metrics.mean_absolute_error(y_test, svr_pred))
print('MSE:', metrics.mean_squared_error(y_test, svr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, svr_pred)))
print('r2 score:',r2)

R2_Scores.append(r2)

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_test,svr_pred, c='red')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.show()

# RandomForestRegressor


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 1500, random_state = 0)
rfr.fit(X_train, y_train)

In [ ]:
rfr_pred= rfr.predict(X_test)
rfr_pred = rfr_pred.reshape(-1,1)

In [ ]:
r2=r2_score(y_test, rfr_pred)
print('MAE:', metrics.mean_absolute_error(y_test, rfr_pred))
print('MSE:', metrics.mean_squared_error(y_test, rfr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, rfr_pred)))
print('r2 score:',r2)

R2_Scores.append(r2)

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_test,rfr_pred, c='orange')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.show()

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
clf_knn = KNeighborsRegressor()
clf_knn.fit(X_train , y_train)
accuracies = cross_val_score(estimator = clf_knn, X = X_train, y = y_train, cv = 5,verbose = 1)
y_pred = clf_knn.predict(X_test)
print('')
print('###### KNeighbours Regression ######')
print('Score : %.4f' % clf_knn.score(X_test, y_test))
print(accuracies)

mse = metrics.mean_squared_error(y_test, y_pred)
mae = metrics.mean_absolute_error(y_test, y_pred)
rmse = metrics.mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)

print('')
print('MSE    : %0.2f ' % mse)
print('MAE    : %0.2f ' % mae)
print('RMSE   : %0.2f ' % rmse)
print('R2     : %0.2f ' % r2)

In [ ]:
from sklearn.model_selection import GridSearchCV
n_neighbors=[]
for i in range (0,50,5):
    if(i!=0):
        n_neighbors.append(i)
params_dict={'n_neighbors':n_neighbors,'n_jobs':[-1]}
clf_knn=GridSearchCV(estimator=KNeighborsRegressor(),param_grid=params_dict,scoring='r2')
clf_knn.fit(X_train,y_train)
print('Score : %.4f' % clf_knn.score(X_test, y_test))
pred=clf_knn.predict(X_test)
r2 = r2_score(y_test, pred)
print('R2     : %0.2f ' % r2)
R2_Scores.append(r2)

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_test,pred, c='orange')
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.show()